# 1. Summary

This notebook documents the methods for computing the ambient color from a camera frame.

# 2. Get Image to Process

Use either a frame from the camera or the example image to run through the color algorithm examples in Section 3.

## 2.1 (Options 1) Use Camera Frame

In [ ]:
import cv2

def show_img(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)

cap = cv2.VideoCapture(0)
ret, img = cap.read()

show_img(img)

## 2.2 (Option 2) Use Example Image

In [ ]:
img = cv2.imread('obama.jpg')

show_img(img)

# 3. Ambient Color Algorithms

This section will define the various methods that can be used to determine the ambient color from an image.

## 3.1 Average Pixel Color

This method of finding the ambient color simply involves averaging the RGB values of the frame and then converting it to a CCT value.

### 3.1.1 (Step 1) Average Pixel Value

Find the average pixel value by taking the mean of all the colors in the image.

In [ ]:
import numpy as np

def avg_image_colors(img):
    average = img.mean(axis=0).mean(axis=0)
    avg_img = np.ones(shape=img.shape, dtype=np.uint8)*np.uint8(average)
    return avg_img[4, 4]

def bgr_to_rgb(color_bgr):
    return (color_bgr[2], color_bgr[1], color_bgr[0])


def show_color(color_bgr):
    color_rgb = bgr_to_rgb(color_bgr)
    plt.imshow([[color_rgb]])
    
show_color(avg_image_colors(img))

### 3.1.2 (Step 2) Convert Average Pixel Value to CCT Value

Convert the mean color into a CCT value to prevent extreme adaptation targets.

In [ ]:
import colour
from colour.plotting import plot_planckian_locus_in_chromaticity_diagram_CIE1960UCS

def bgr_to_cct(color_bgr):
    print("Mean RGB Value: {}".format(color_bgr))

    avg_img_color_xyz = colour.sRGB_to_XYZ(color_bgr / 255)
    avg_img_color_xy = colour.XYZ_to_xy(avg_img_color_xyz)
    avg_img_color_cct = colour.xy_to_CCT(avg_img_color_xy, 'hernandez1999')

    print('Mean CCT Value: {}'.format(avg_img_color_cct))

    avg_img__color_cct_xy = colour.temperature.CCT_to_xy(avg_img_color_cct)
    avg_img_color_cct_xyz = colour.xy_to_XYZ(avg_img__color_cct_xy)
    return colour.XYZ_to_sRGB(avg_img_color_cct_xyz)
        
plt.imshow([[bgr_to_cct(avg_image_colors(img))]])

## 3.1.3 Analysis

Finding the ambient color through average all the pixels values is clearly not the right solution.
The heuristic will almost always generate a color that is not representative or even present in the
image - it takes into account irrelevant objects like the user's face or background objects.

The step to convert the average color to CCT helps clamps the color to an acceptable range, but again
the target color is wrong so the CCT value will also be wrong.

## 3.2 K-Means Color Clusters

This method attempts to improve on the previous one by being more intelligent on the colors it chooses to use for averaging.
The assumption of this heuristic is the background takes up the majority of the camera frame. This means if the pixel values are
clustered into similar colors then the largest cluster will represent the background color.

### 3.2.1 (Step 1) Apply K-Means Cluster

Note: This algorithm can possibly be improved by averaging all the colors in the most populous cluster to get a
better representation, but that is skipped since this is still clearly the wrong solution.

In [ ]:
def apply_kmeans_img(img, k, debug=False):

    pixels = np.float32(img.reshape(-1, 3))

    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 50, .1)
    flags = cv2.KMEANS_RANDOM_CENTERS

    _, labels, centroids = cv2.kmeans(pixels, k, None, criteria, 10, flags)
    centroid = centroids[0] / 255

    if debug:
        print(centroid)
    return centroid

bgr_value_k_1 = apply_kmeans_img(img, 1, True)
bgr_value_k_5 = apply_kmeans_img(img, 5, True)
bgr_value_k_10 = apply_kmeans_img(img, 10, True)

f, axarr = plt.subplots(1, 3)
axarr[0].imshow([[bgr_to_rgb(bgr_value_k_1)]])
axarr[1].imshow([[bgr_to_rgb(bgr_value_k_5)]])
axarr[2].imshow([[bgr_to_rgb(bgr_value_k_10)]])

### Step 3.2.2 (Step 2) Convert K-Means Color To CCT

In [ ]:
f, axarr = plt.subplots(1, 3)
axarr[0].imshow([[bgr_to_cct(bgr_value_k_1 * 255)]])
axarr[1].imshow([[bgr_to_cct(bgr_value_k_5 * 255)]])
axarr[2].imshow([[bgr_to_cct(bgr_value_k_10 * 255)]])

### 3.2.3 Analysis

Using K-Means to find the ambient color of the image is also an incorrect heuristic as the dominant color may not always be the background.

It is clear the ambient color cannot be produced from a camera frame by using any sort of color averaging.

# 3.3 Use ML to Find Ambient Color

This is inspired by augmented reality SDKs which are able to calculate the ambient color of a scene.

For example ARCore is able to calculate the lighting of a scene and compensate virtual objects:

![ar_example](ar_example.png)

The SDK exposes a [getColorCorrection()](https://developers.google.com/ar/reference/java/arcore/reference/com/google/ar/core/LightEstimate#getColorCorrection(float[],%20int)) call which can be used to find the ambient color.

This is currently not a possible option since there are no augmented reality SDKs for Windows desktop.